## Timeseries clustering

Time series clustering is to partition time series data into groups based on similarity or distance, so that time series in the same cluster are similar.

Methodology followed:
* Use Variational Recurrent AutoEncoder (VRAE) for dimensionality reduction of the timeseries
* To visualize the clusters, PCA and t-sne are used

Paper:
https://arxiv.org/pdf/1412.6581.pdf

#### Contents

0. [Load data and preprocess](#Load-data-and-preprocess)
1. [Initialize VRAE object](#Initialize-VRAE-object)
2. [Fit the model onto dataset](#Fit-the-model-onto-dataset)
3. [Transform the input timeseries to encoded latent vectors](#Transform-the-input-timeseries-to-encoded-latent-vectors)
4. [Save the model to be fetched later](#Save-the-model-to-be-fetched-later)
5. [Visualize using PCA and tSNE](#Visualize-using-PCA-and-tSNE)

In [ ]:
# from IPython.display import HTML
# HTML('''<script>
# code_show=true; 
# function code_toggle() {
# if (code_show){
# $('div.input').hide();
# } else {
# $('div.input').show();
# }
# code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

### Import required modules

In [ ]:
import copy
import numpy as np
import pandas as pd

from vrae.vrae import VRAE
from vrae.utils import *
import torch
from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, confusion_matrix

from tensorflow.keras.utils import to_categorical

In [ ]:
gpu_id = 2

if gpu_id>=0:
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    cuda_id = "cuda:" + str(0)  # cuda:2

device = torch.device(cuda_id if torch.cuda.is_available() else "cpu")
print("Device:", device)
if (torch.cuda.is_available()):
    torch.cuda.set_device(cuda_id)
    print("Current GPU ID:", torch.cuda.current_device())

### Input parameters

In [ ]:
dload = './model_dir' #download directory

### Hyper parameters

In [ ]:
hidden_size = 120 #90
hidden_layer_depth = 1
latent_length = 80 # 40 20
batch_size = 64
learning_rate = 0.0005 # 0.0005
n_epochs = 800
dropout_rate = 0
optimizer = 'Adam' # options: ADAM, SGD
cuda = True # options: True, False
print_every=30
clip = False # options: True, False
max_grad_norm=5
loss = 'MSELoss' # options: SmoothL1Loss, MSELoss
block = 'LSTM' # options: LSTM, GRU

corr = 'Gaussian' # options: Gaussian, ZeroMask, ConsecutiveZeros, Both
sigma = 0.3 # 0.1
if corr == 'ConsecutiveZeros':
    sigma = [60, 80] # [lambda_corr, lambda_norm]
if corr == 'Both':
    sigma = [0.1, 50, 80]

seed = 0


In [ ]:
# random.seed(args.random_seed)
# generator = torch.Generator()
# generator.manual_seed(seed)

np.random.seed(seed)
torch.manual_seed(seed)

### Load data and preprocess

In [ ]:
window_len = 512 # 512
stride_len = 20 # 100
act_list = [1, 2, 3, 4, 5, 6, 7, 12, 13, 16, 17, 24]
# act_list = [1, 2]
act_labels_txt = ['lay', 'sit', 'std', 'wlk', 'run', 'cyc', 'nord', 'ups', 'dws', 'vac', 'iron', 'rop']

In [ ]:
X=[]
user_labels=[]
act_labels=[]

# columns for IMU data
# 4-20 IMU hand
# 21-37 IMU chest
# 38-54 IMU ankle
# 2-4 3D-acceleration data (ms-2), scale: ±16g, resolution: 13-bit
# 8-10 3D-gyroscope data (rad/s)
# 11-13 3D-magnetometer data (μT)
imu_locs = [4,5,6, 10,11,12, 13,14,15, 
            21,22,23, 27,28,29, 30,31,32, 
            38,39,40, 44,45,46, 47,48,49
           ] 

# scaler = StandardScaler()

for uid in np.arange(1,10):
    path = '../../PAMAP2_Dataset/Protocol/subject10' + str(uid) + '.dat'
    df = pd.read_table(path, sep=' ', header=None)
    act_imu_filter = df.iloc[:, imu_locs] 


    for act_id in range(len(act_list)):
        act_filter =  act_imu_filter[df.iloc[:, 1] == act_list[act_id]]
        act_data = act_filter.to_numpy()
        act_data = np.transpose(act_data)

        # sliding window segmentation
        start_idx = 0
        while start_idx + window_len < act_data.shape[1]:
            window_data = act_data[:, start_idx:start_idx+window_len]
            downsamp_data = window_data[:, ::3] # downsample from 100hz to 33.3hz
            downsamp_data = np.nan_to_num(downsamp_data) # remove nan
            # downsamp_data = np.transpose(downsamp_data) # dim: seq_len, feature_len

            X.append(downsamp_data)
            user_labels.append(uid)
            act_labels.append(act_id)
            start_idx = start_idx + stride_len
            
X = np.array(X).astype('float32')

In [ ]:
normalized_X = np.zeros_like(X)
for ch_id in range(X.shape[1]):
    ch_data = X[:, ch_id, :]
    scaler = MinMaxScaler()
    ch_data = scaler.fit_transform(ch_data)
    normalized_X[:, ch_id, :] = ch_data
X = np.transpose(normalized_X, (0, 2, 1)) # num_samples, sequence_length, feature_length

In [ ]:
# X = X.reshape(X.shape[0], 1, X.shape[1], X.shape[2]) # convert list to numpy array
act_labels = np.array(act_labels).astype('float32')
act_labels = act_labels.reshape(act_labels.shape[0],1)
act_labels = to_categorical(act_labels, num_classes=len(act_list))

In [ ]:
print(X.shape)
print(act_labels.shape)

In [ ]:
dataset = TensorDataset(torch.from_numpy(X), torch.from_numpy(act_labels))

# Train/Test dataset split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

trainLoader = torch.utils.data.DataLoader(train_dataset,
    batch_size=batch_size, shuffle=True) 

testLoader = torch.utils.data.DataLoader(test_dataset,
    batch_size=batch_size, shuffle=False)

In [ ]:
sequence_length = X.shape[1]
number_of_features = X.shape[2]

In [ ]:
print(sequence_length)
print(number_of_features)

In [ ]:
import numpy as np
import torch
from torch import nn, optim
from torch import distributions
from vrae.base import BaseEstimator
from torch.utils.data import DataLoader
from torch.autograd import Variable
import os


class Encoder(nn.Module):
    """
    Encoder network containing enrolled LSTM/GRU

    :param number_of_features: number of input features
    :param hidden_size: hidden size of the RNN
    :param hidden_layer_depth: number of layers in RNN
    :param latent_length: latent vector length
    :param dropout: percentage of nodes to dropout
    :param block: LSTM/GRU block
    """
    def __init__(self, number_of_features, hidden_size, hidden_layer_depth, latent_length, dropout, corr, sigma, block = 'LSTM'):

        super(Encoder, self).__init__()

        self.number_of_features = number_of_features
        self.hidden_size = hidden_size
        self.hidden_layer_depth = hidden_layer_depth
        self.latent_length = latent_length
        self.corr = corr
        self.sigma = sigma

        if block == 'LSTM':
            self.model = nn.LSTM(self.number_of_features, self.hidden_size, self.hidden_layer_depth, dropout = dropout)
        elif block == 'GRU':
            self.model = nn.GRU(self.number_of_features, self.hidden_size, self.hidden_layer_depth, dropout = dropout)
        else:
            raise NotImplementedError
            
    def corrupt(self, x):
        if self.corr == 'Gaussian':
            noise = self.sigma * torch.randn(x.size()).type_as(x)
            return x + noise
        elif self.corr == 'ZeroMask':
            num_zeros = int(torch.numel(x) * self.sigma)
            mask = torch.ones(torch.numel(x), device=x.device)
            mask[:num_zeros] = 0
            mask = mask[torch.randperm(mask.shape[0])]
            mask = mask.reshape(x.shape)
            return torch.mul(x, mask)
        elif self.corr == 'ConsecutiveZeros':
            # time * batch_size * feature
            # lambdas reuse the sigma variable, unpack
            lambda_corr = self.sigma[0] # lambda for missing data period
            lambda_norm = self.sigma[1] # lambda for normal data periodß
            mask = torch.ones_like(x)
            for sample_id in range(mask.shape[1]):
                for ch_id in range(mask.shape[2]):
                    ptr = 0
                    is_corrupted = False
                    while ptr < mask.shape[0]:
                        if is_corrupted:
                            corr_duration = int(np.random.exponential(scale=lambda_corr))
                            mask[ptr:min(mask.shape[0], ptr + corr_duration), sample_id, ch_id] = 0
                            ptr = min(mask.shape[0], ptr + corr_duration)
                            is_corrupted = False
                        else:
                            norm_duration = int(np.random.exponential(scale=lambda_norm))
                            ptr = min(mask.shape[0], ptr + norm_duration)
                            is_corrupted = True
            return torch.mul(x, mask)
        elif self.corr == 'Both':
            noise = self.sigma[0] * torch.randn(x.size()).type_as(x)
            # time * batch_size * feature
            # lambdas reuse the sigma variable, unpack
            lambda_corr = self.sigma[1] # lambda for missing data period
            lambda_norm = self.sigma[2] # lambda for normal data periodß
            mask = torch.ones_like(x)
            for sample_id in range(mask.shape[1]):
                for ch_id in range(mask.shape[2]):
                    ptr = 0
                    is_corrupted = False
                    while ptr < mask.shape[0]:
                        if is_corrupted:
                            corr_duration = int(np.random.exponential(scale=lambda_corr))
                            mask[ptr:min(mask.shape[0], ptr + corr_duration), sample_id, ch_id] = 0
                            ptr = min(mask.shape[0], ptr + corr_duration)
                            is_corrupted = False
                        else:
                            norm_duration = int(np.random.exponential(scale=lambda_norm))
                            ptr = min(mask.shape[0], ptr + norm_duration)
                            is_corrupted = True
            return torch.mul((x + noise), mask)            
        else:
            raise NotImplementedError

    def forward(self, x):
        """Forward propagation of encoder. Given input, outputs the last hidden state of encoder

        :param x: input to the encoder, of shape (sequence_length, batch_size, number_of_features)
        :return: last hidden state of encoder, of shape (batch_size, hidden_size)
        """
        
         # _, (h_end, c_end) = self.model(x)
        # Adding corruption
        x_corr = self.corrupt(x)
        _, (h_end, c_end) = self.model(x_corr)

        h_end = h_end[-1, :, :]
        return h_end
    
    def forward_corr_x(self, x_corr):
        """Forward propagation of encoder. Given input, outputs the last hidden state of encoder

        :param x: input to the encoder, of shape (sequence_length, batch_size, number_of_features)
        :return: last hidden state of encoder, of shape (batch_size, hidden_size)
        """
        
         # _, (h_end, c_end) = self.model(x)
        # Adding corruption
        # x_corr = self.corrupt(x)
        _, (h_end, c_end) = self.model(x_corr)

        h_end = h_end[-1, :, :]
        return h_end


class Lambda(nn.Module):
    """Lambda module converts output of encoder to latent vector

    :param hidden_size: hidden size of the encoder
    :param latent_length: latent vector length
    """
    def __init__(self, hidden_size, latent_length):
        super(Lambda, self).__init__()

        self.hidden_size = hidden_size
        self.latent_length = latent_length

        self.hidden_to_mean = nn.Linear(self.hidden_size, self.latent_length)
        self.hidden_to_logvar = nn.Linear(self.hidden_size, self.latent_length)

        nn.init.xavier_uniform_(self.hidden_to_mean.weight)
        nn.init.xavier_uniform_(self.hidden_to_logvar.weight)

    def forward(self, cell_output):
        """Given last hidden state of encoder, passes through a linear layer, and finds the mean and variance

        :param cell_output: last hidden state of encoder
        :return: latent vector
        """
        # print(self.training)

        self.latent_mean = self.hidden_to_mean(cell_output)
        self.latent_logvar = self.hidden_to_logvar(cell_output)
        
        # std = torch.exp(0.5 * self.latent_logvar)
        # eps = torch.randn_like(std)
        # return eps.mul(std).add_(self.latent_mean)
    
        if self.training:
            std = torch.exp(0.5 * self.latent_logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(self.latent_mean)
        else:
            return self.latent_mean

class Decoder(nn.Module):
    """Converts latent vector into output

    :param sequence_length: length of the input sequence
    :param batch_size: batch size of the input sequence
    :param hidden_size: hidden size of the RNN
    :param hidden_layer_depth: number of layers in RNN
    :param latent_length: latent vector length
    :param output_size: 2, one representing the mean, other log std dev of the output
    :param block: GRU/LSTM - use the same which you've used in the encoder
    :param dtype: Depending on cuda enabled/disabled, create the tensor
    """
    def __init__(self, sequence_length, batch_size, hidden_size, hidden_layer_depth, latent_length, output_size, dtype, block='LSTM'):

        super(Decoder, self).__init__()

        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self.hidden_layer_depth = hidden_layer_depth
        self.latent_length = latent_length
        self.output_size = output_size
        self.dtype = dtype

        if block == 'LSTM':
            self.model = nn.LSTM(1, self.hidden_size, self.hidden_layer_depth)
        elif block == 'GRU':
            self.model = nn.GRU(1, self.hidden_size, self.hidden_layer_depth)
        else:
            raise NotImplementedError

        self.latent_to_hidden = nn.Linear(self.latent_length, self.hidden_size)
        self.hidden_to_output = nn.Linear(self.hidden_size, self.output_size)

        self.decoder_inputs = torch.zeros(self.sequence_length, self.batch_size, 1, requires_grad=True).type(self.dtype)
        self.c_0 = torch.zeros(self.hidden_layer_depth, self.batch_size, self.hidden_size, requires_grad=True).type(self.dtype)

        nn.init.xavier_uniform_(self.latent_to_hidden.weight)
        nn.init.xavier_uniform_(self.hidden_to_output.weight)

    def forward(self, latent):
        """Converts latent to hidden to output

        :param latent: latent vector
        :return: outputs consisting of mean and std dev of vector
        """
        h_state = self.latent_to_hidden(latent)

        if isinstance(self.model, nn.LSTM):
            h_0 = torch.stack([h_state for _ in range(self.hidden_layer_depth)])
            decoder_output, _ = self.model(self.decoder_inputs, (h_0, self.c_0))
        elif isinstance(self.model, nn.GRU):
            h_0 = torch.stack([h_state for _ in range(self.hidden_layer_depth)])
            decoder_output, _ = self.model(self.decoder_inputs, h_0)
        else:
            raise NotImplementedError

        out = self.hidden_to_output(decoder_output)
        return out

def _assert_no_grad(tensor):
    assert not tensor.requires_grad, \
        "nn criterions don't compute the gradient w.r.t. targets - please " \
        "mark these tensors as not requiring gradients"

class VRAE(BaseEstimator, nn.Module):
    """Variational recurrent auto-encoder. This module is used for dimensionality reduction of timeseries

    :param sequence_length: length of the input sequence
    :param number_of_features: number of input features
    :param hidden_size:  hidden size of the RNN
    :param hidden_layer_depth: number of layers in RNN
    :param latent_length: latent vector length
    :param batch_size: number of timeseries in a single batch
    :param learning_rate: the learning rate of the module
    :param block: GRU/LSTM to be used as a basic building block
    :param n_epochs: Number of iterations/epochs
    :param dropout_rate: The probability of a node being dropped-out
    :param optimizer: ADAM/ SGD optimizer to reduce the loss function
    :param loss: SmoothL1Loss / MSELoss / ReconLoss / any custom loss which inherits from `_Loss` class
    :param boolean cuda: to be run on GPU or not
    :param print_every: The number of iterations after which loss should be printed
    :param boolean clip: Gradient clipping to overcome explosion
    :param max_grad_norm: The grad-norm to be clipped
    :param dload: Download directory where models are to be dumped
    """
    def __init__(self, sequence_length, number_of_features, corr, sigma, hidden_size=90, hidden_layer_depth=2, latent_length=20,
                 batch_size=32, learning_rate=0.005, block='LSTM',
                 n_epochs=5, dropout_rate=0., optimizer='Adam', loss='MSELoss',
                 cuda=False, print_every=100, clip=True, max_grad_norm=5, dload='.'):

        super(VRAE, self).__init__()


        self.dtype = torch.FloatTensor
        self.use_cuda = cuda

        if not torch.cuda.is_available() and self.use_cuda:
            self.use_cuda = False


        if self.use_cuda:
            self.dtype = torch.cuda.FloatTensor


        self.encoder1 = Encoder(number_of_features = int(number_of_features/3),
                               hidden_size=hidden_size,
                               hidden_layer_depth=hidden_layer_depth,
                               latent_length=latent_length,
                               dropout=dropout_rate,
                               corr=corr,
                               sigma=sigma,
                               block=block)

        self.encoder2 = Encoder(number_of_features = int(number_of_features/3),
                               hidden_size=hidden_size,
                               hidden_layer_depth=hidden_layer_depth,
                               latent_length=latent_length,
                               dropout=dropout_rate,
                               corr=corr,
                               sigma=sigma,
                               block=block)
        
        self.encoder3 = Encoder(number_of_features = int(number_of_features/3),
                               hidden_size=hidden_size,
                               hidden_layer_depth=hidden_layer_depth,
                               latent_length=latent_length,
                               dropout=dropout_rate,
                               corr=corr,
                               sigma=sigma,
                               block=block)
        
        self.lmbd1 = Lambda(hidden_size=hidden_size,
                           latent_length=latent_length)
        
        self.lmbd2 = Lambda(hidden_size=hidden_size,
                           latent_length=latent_length)
        
        self.lmbd3 = Lambda(hidden_size=hidden_size,
                           latent_length=latent_length)        
        
        self.decoder1 = Decoder(sequence_length=sequence_length,
                               batch_size = batch_size,
                               hidden_size=hidden_size,
                               hidden_layer_depth=hidden_layer_depth,
                               latent_length=latent_length,
                               output_size=int(number_of_features/3),
                               block=block,
                               dtype=self.dtype)
        
        self.decoder2 = Decoder(sequence_length=sequence_length,
                               batch_size = batch_size,
                               hidden_size=hidden_size,
                               hidden_layer_depth=hidden_layer_depth,
                               latent_length=latent_length,
                               output_size=int(number_of_features/3),
                               block=block,
                               dtype=self.dtype)
        
        self.decoder3 = Decoder(sequence_length=sequence_length,
                               batch_size = batch_size,
                               hidden_size=hidden_size,
                               hidden_layer_depth=hidden_layer_depth,
                               latent_length=latent_length,
                               output_size=int(number_of_features/3),
                               block=block,
                               dtype=self.dtype)        

        self.sequence_length = sequence_length
        self.hidden_size = hidden_size
        self.hidden_layer_depth = hidden_layer_depth
        self.latent_length = latent_length
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs

        self.print_every = print_every
        self.clip = clip
        self.max_grad_norm = max_grad_norm
        self.is_fitted = False
        self.dload = dload

        if self.use_cuda:
            self.cuda()

        if optimizer == 'Adam':
            self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        elif optimizer == 'SGD':
            self.optimizer = optim.SGD(self.parameters(), lr=learning_rate)
        else:
            raise ValueError('Not a recognized optimizer')

        if loss == 'SmoothL1Loss':
            self.loss_fn = nn.SmoothL1Loss(size_average=False)
                        
        elif loss == 'MSELoss':
            self.loss_fn = nn.MSELoss(size_average=False)

    def __repr__(self):
        return """VRAE(n_epochs={n_epochs},batch_size={batch_size},cuda={cuda})""".format(
                n_epochs=self.n_epochs,
                batch_size=self.batch_size,
                cuda=self.use_cuda)

    def forward(self, x):
        """
        Forward propagation which involves one pass from inputs to encoder to lambda to decoder

        :param x:input tensor
        :return: the decoded output, latent vector
        """
        # cell_output = self.encoder(x)
        # latent = self.lmbd(cell_output)
        # x_decoded = self.decoder(latent)
        
        # x: torch.Size([171, 64, 27])

        x1 = x[:,:, [0,1,2, 9,10,11, 18,19,20]] # acc
        x2 = x[:,:, [3,4,5, 12,13,14, 21,22,23]] # gyro
        x3 = x[:,:, [6,7,8, 15,16,17, 24,25,26]] # mag

        
        cell_output1 = self.encoder1(x1)
        cell_output2 = self.encoder2(x2)
        cell_output3 = self.encoder3(x3)

        latent1 = self.lmbd1(cell_output1)
        latent2 = self.lmbd2(cell_output2)
        latent3 = self.lmbd3(cell_output3)
        
        
        x_decoded1 = self.decoder1(latent1)
        x_decoded2 = self.decoder2(latent2)        
        x_decoded3 = self.decoder3(latent3)
        
#         x_decoded = torch.zeros_like(x)
        
#         x_decoded[:,:, [0,1,2, 9,10,11, 18,19,20]] = x_decoded1 # acc
#         x_decoded[:,:, [3,4,5, 12,13,14, 21,22,23]] = x_decoded2 # gyro
#         x_decoded[:,:, [6,7,8, 15,16,17, 24,25,26]] = x_decoded3 # mag
        
        return x_decoded1, x_decoded2, x_decoded3, latent1, latent2, latent3

    def soft_hgr(self, f_x, g_y):
        """Computes soft-HGR objective"""
        num_samps = f_x.shape[0]

        f_x_new = f_x - f_x.mean(dim=0)
        cov_f = torch.mm(torch.t(f_x_new), f_x_new) / (num_samps-1)

        g_y_new = g_y - g_y.mean(dim=0)
        cov_g = torch.mm(torch.t(g_y_new), g_y_new) / (num_samps-1)

        hgr_loss = -torch.trace(torch.mm(torch.t(f_x_new), g_y_new) / (num_samps-1)) + 0.5*torch.trace(torch.mm(cov_f, cov_g))

        return hgr_loss    
    
    def _rec(self, x_decoded1, x_decoded2, x_decoded3, latent1, latent2, latent3, x, loss_fn):
        """
        Compute the loss given output x decoded, input x and the specified loss function

        :param x_decoded: output of the decoder
        :param x: input to the encoder
        :param loss_fn: loss function specified
        :return: joint loss, reconstruction loss and kl-divergence loss
        """
        # latent_mean, latent_logvar = self.lmbd.latent_mean, self.lmbd.latent_logvar
        
        latent_mean1, latent_logvar1 = self.lmbd1.latent_mean, self.lmbd1.latent_logvar
        latent_mean2, latent_logvar2 = self.lmbd2.latent_mean, self.lmbd2.latent_logvar
        latent_mean3, latent_logvar3 = self.lmbd3.latent_mean, self.lmbd3.latent_logvar
        
        
        hgr_loss = self.soft_hgr(latent1, latent2) + self.soft_hgr(latent1, latent3) + self.soft_hgr(latent2, latent3)

        kl_loss1 = -0.5 * torch.mean(1 + latent_logvar1 - latent_mean1.pow(2) - latent_logvar1.exp())
        kl_loss2 = -0.5 * torch.mean(1 + latent_logvar2 - latent_mean2.pow(2) - latent_logvar2.exp())
        kl_loss3 = -0.5 * torch.mean(1 + latent_logvar3 - latent_mean3.pow(2) - latent_logvar3.exp())
        
        kl_loss = kl_loss1 + kl_loss2 + kl_loss3
        # kl_loss = -0.5 * torch.mean(1 + latent_logvar - latent_mean.pow(2) - latent_logvar.exp())

        x1 = x[:,:, [0,1,2, 9,10,11, 18,19,20]] # acc
        x2 = x[:,:, [3,4,5, 12,13,14, 21,22,23]] # gyro
        x3 = x[:,:, [6,7,8, 15,16,17, 24,25,26]] # mag
        
        recon_loss1 = loss_fn(x_decoded1, x1)
        recon_loss2 = loss_fn(x_decoded2, x2)
        recon_loss3 = loss_fn(x_decoded3, x3)
        
        recon_loss = recon_loss1 + recon_loss2 + recon_loss3
        # recon_loss = loss_fn(x_decoded, x)

        return kl_loss + recon_loss + hgr_loss, recon_loss, kl_loss, hgr_loss

    def compute_loss(self, X):
        """
        Given input tensor, forward propagate, compute the loss, and backward propagate.
        Represents the lifecycle of a single iteration

        :param X: Input tensor
        :return: total loss, reconstruction loss, kl-divergence loss and original input
        """
        x = Variable(X[:,:,:].type(self.dtype), requires_grad = True)

        x_decoded1, x_decoded2, x_decoded3, latent1, latent2, latent3 = self(x)
        # x_decoded, _ = self(x)

        loss, recon_loss, kl_loss, hgr_loss = self._rec(x_decoded1, x_decoded2, x_decoded3, latent1, latent2, latent3, x.detach(), self.loss_fn)

        return loss, recon_loss, kl_loss, hgr_loss, x


    def _train(self, train_loader):
        """
        For each epoch, given the batch_size, run this function batch_size * num_of_batches number of times

        :param train_loader:input train loader with shuffle
        :return:
        """
        self.train()

        epoch_loss = 0
        t = 0

        for t, X in enumerate(train_loader):

            # Index first element of array to return tensor
            X = X[0]

            # required to swap axes, since dataloader gives output in (batch_size x seq_len x num_of_features)
            X = X.permute(1,0,2)

            self.optimizer.zero_grad()
            loss, recon_loss, kl_loss, hgr_loss, _ = self.compute_loss(X)
            loss.backward()

            if self.clip:
                torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm = self.max_grad_norm)

            # accumulator
            epoch_loss += loss.item()

            self.optimizer.step()

            if (t + 1) % self.print_every == 0:
                print('Batch %d, loss = %.4f, recon_loss = %.4f, kl_loss = %.4f, hgr_loss = %.4f' % (t + 1, loss.item(),
                                                                                    recon_loss.item(), kl_loss.item(), hgr_loss.item()))

        print('Average loss: {:.4f}'.format(epoch_loss / t))


    def fit(self, dataset, save = False):
        """
        Calls `_train` function over a fixed number of epochs, specified by `n_epochs`

        :param dataset: `Dataset` object
        :param bool save: If true, dumps the trained model parameters as pickle file at `dload` directory
        :return:
        """

        train_loader = DataLoader(dataset = dataset,
                                  batch_size = self.batch_size,
                                  shuffle = True,
                                  drop_last=True)

        for i in range(self.n_epochs):
            print('Epoch: %s' % i)

            self._train(train_loader)

        self.is_fitted = True
        if save:
            self.save('model.pth')


    def _batch_transform(self, x):
        """
        Passes the given input tensor into encoder and lambda function

        :param x: input batch tensor
        :return: intermediate latent vector
        """
        return self.lmbd(
                    self.encoder(
                        Variable(x.type(self.dtype), requires_grad = False)
                    )
        ).cpu().data.numpy()

    def _batch_reconstruct(self, x):
        """
        Passes the given input tensor into encoder, lambda and decoder function

        :param x: input batch tensor
        :return: reconstructed output tensor
        """

        x = Variable(x.type(self.dtype), requires_grad = False)
        x_decoded, _ = self(x)

        return x_decoded.cpu().data.numpy()
    
    
    def _batch_reconstruct_corr_x(self, x):
        """
        Passes the given input tensor into encoder, lambda and decoder function

        :param x: input batch tensor
        :return: reconstructed output tensor
        """

        x = Variable(x.type(self.dtype), requires_grad = False)
        # x_decoded, _ = self(x)
        
        x1 = x[:,:, [0,1,2, 9,10,11, 18,19,20]] # acc
        x2 = x[:,:, [3,4,5, 12,13,14, 21,22,23]] # gyro
        x3 = x[:,:, [6,7,8, 15,16,17, 24,25,26]] # mag
        
        cell_output1 = self.encoder1.forward_corr_x(x1)
        cell_output2 = self.encoder2.forward_corr_x(x2)
        cell_output3 = self.encoder3.forward_corr_x(x3)
        
        latent1 = self.lmbd1(cell_output1)
        latent2 = self.lmbd2(cell_output2)
        latent3 = self.lmbd3(cell_output3)
        
        
        x_decoded1 = self.decoder1(latent1)
        x_decoded2 = self.decoder2(latent2)
        x_decoded3 = self.decoder3(latent3)
        
        x_decoded = torch.zeros_like(x)
        
        x_decoded[:,:, [0,1,2, 9,10,11, 18,19,20]] = x_decoded1 # acc
        x_decoded[:,:, [3,4,5, 12,13,14, 21,22,23]] = x_decoded2 # gyro
        x_decoded[:,:, [6,7,8, 15,16,17, 24,25,26]] = x_decoded3 # mag        

        return x_decoded.cpu().data.numpy()

    def reconstruct(self, dataset, save = False):
        """
        Given input dataset, creates dataloader, runs dataloader on `_batch_reconstruct`
        Prerequisite is that model has to be fit

        :param dataset: input dataset who's output vectors are to be obtained
        :param bool save: If true, dumps the output vector dataframe as a pickle file
        :return:
        """

        self.eval()

        test_loader = DataLoader(dataset = dataset,
                                 batch_size = self.batch_size,
                                 shuffle = False,
                                 drop_last=True) # Don't shuffle for test_loader

        if self.is_fitted:
            with torch.no_grad():
                x_decoded = []

                for t, x in enumerate(test_loader):
                    x = x[0]
                    x = x.permute(1, 0, 2)
                    # x_decoded_each = self._batch_reconstruct(x)

                    x_decoded_each = self._batch_reconstruct_corr_x(x)
                    x_decoded.append(x_decoded_each)

                x_decoded = np.concatenate(x_decoded, axis=1)

                if save:
                    if os.path.exists(self.dload):
                        pass
                    else:
                        os.mkdir(self.dload)
                    x_decoded.dump(self.dload + '/z_run.pkl')
                return x_decoded

        raise RuntimeError('Model needs to be fit')


    def transform(self, dataset, save = False):
        """
        Given input dataset, creates dataloader, runs dataloader on `_batch_transform`
        Prerequisite is that model has to be fit

        :param dataset: input dataset who's latent vectors are to be obtained
        :param bool save: If true, dumps the latent vector dataframe as a pickle file
        :return:
        """
        self.eval()

        test_loader = DataLoader(dataset = dataset,
                                 batch_size = self.batch_size,
                                 shuffle = False,
                                 drop_last=True) # Don't shuffle for test_loader
        if self.is_fitted:
            with torch.no_grad():
                z_run = []

                for t, x in enumerate(test_loader):
                    x = x[0]
                    x = x.permute(1, 0, 2)

                    z_run_each = self._batch_transform(x)
                    z_run.append(z_run_each)

                z_run = np.concatenate(z_run, axis=0)
                if save:
                    if os.path.exists(self.dload):
                        pass
                    else:
                        os.mkdir(self.dload)
                    z_run.dump(self.dload + '/z_run.pkl')
                return z_run

        raise RuntimeError('Model needs to be fit')
        

    def fit_transform(self, dataset, save = False):
        """
        Combines the `fit` and `transform` functions above

        :param dataset: Dataset on which fit and transform have to be performed
        :param bool save: If true, dumps the model and latent vectors as pickle file
        :return: latent vectors for input dataset
        """
        self.fit(dataset, save = save)
        return self.transform(dataset, save = save)

    def save(self, file_name):
        """
        Pickles the model parameters to be retrieved later

        :param file_name: the filename to be saved as,`dload` serves as the download directory
        :return: None
        """
        PATH = self.dload + '/' + file_name
        if os.path.exists(self.dload):
            pass
        else:
            os.mkdir(self.dload)
        torch.save(self.state_dict(), PATH)

    def load(self, file_name):
        """
        Loads the model's parameters from the path mentioned

        :param PATH: Should contain pickle file
        :return: None
        """
        PATH = self.dload + '/' + file_name
        self.is_fitted = True
        self.load_state_dict(torch.load(PATH))

### Initialize VRAE object

VRAE inherits from `sklearn.base.BaseEstimator` and overrides `fit`, `transform` and `fit_transform` functions, similar to sklearn modules

In [ ]:
vrae = VRAE(sequence_length=sequence_length,
            number_of_features = number_of_features,
            corr = corr,
            sigma = sigma,
            hidden_size = hidden_size, 
            hidden_layer_depth = hidden_layer_depth,
            latent_length = latent_length,
            batch_size = batch_size,
            learning_rate = learning_rate,
            n_epochs = n_epochs,
            dropout_rate = dropout_rate,
            optimizer = optimizer, 
            cuda = cuda,
            print_every=print_every, 
            clip=clip, 
            max_grad_norm=max_grad_norm,
            loss = loss,
            block = block,
            dload = dload)

### Fit the model onto datasettorch.autograd.set_detect_anomaly(True)

In [ ]:
# torch.autograd.set_detect_anomaly(True)

In [ ]:
vrae.fit(train_dataset)

# If the model has to be saved, with the learnt parameters use:
# model saving during training to be implemented, use manual saving in the next block
# vrae.fit(dataset, save = True)

### Save the model to be fetched later

In [ ]:
# Save VRAE model
# sigma = [0.25]
model_name = 'vrae_hgr_mod_h120_l80' + corr
for v in sigma:
    model_name += '_' + str(v)
model_name += '.pth'

vrae.save(model_name)

## Evaluation

In [ ]:
# Load VRAE model from pth file
model_name = 'vrae_hgr_mod' + corr
for v in sigma:
    model_name += '_' + str(v)
model_name += '.pth'

vrae.load(model_name)

In [ ]:
# Prepare testdata set, drop the last incomplete batch
test_x = torch.zeros(len(test_dataset)-35, X.shape[1], X.shape[2])
test_labels = torch.zeros(len(test_dataset)-35, len(act_list))
for test_id in range(len(test_dataset)-35):
    test_labels[test_id] = test_dataset[test_id][1]
    test_x[test_id] = test_dataset[test_id][0]

In [ ]:
test_x1 = test_x[:,:, [0,1,2, 9,10,11, 18,19,20]] # acc
test_x2 = test_x[:,:, [3,4,5, 12,13,14, 21,22,23]] # gyro
test_x3 = test_x[:,:, [6,7,8, 15,16,17, 24,25,26]] # mag

### Corrupt Dataset

In [ ]:
corr_test_x1 = vrae.encoder1.corrupt(test_x1.permute(1,0,2)) # Corruption process is different than used in reconstruction
corr_test_x2 = vrae.encoder2.corrupt(test_x2.permute(1,0,2)) # Corruption process is different than used in reconstruction
corr_test_x3 = vrae.encoder3.corrupt(test_x3.permute(1,0,2)) # Corruption process is different than used in reconstruction

corr_test_x1 = corr_test_x1.permute(1,0,2) # N*171*27
corr_test_x2 = corr_test_x2.permute(1,0,2) # N*171*27
corr_test_x3 = corr_test_x3.permute(1,0,2) # N*171*27

In [ ]:
corr_test_x = torch.zeros_like(test_x)

corr_test_x[:,:, [0,1,2, 9,10,11, 18,19,20]] = corr_test_x1 # acc
corr_test_x[:,:, [3,4,5, 12,13,14, 21,22,23]] = corr_test_x2 # gyro
corr_test_x[:,:, [6,7,8, 15,16,17, 24,25,26]] = corr_test_x3 # mag

### Reconstruct testset (synthesize the entire data)

In [ ]:
corr_test_dataset = TensorDataset(corr_test_x)

In [ ]:
np_recon_test = vrae.reconstruct(corr_test_dataset)

recon_test = torch.from_numpy(np_recon_test).permute(1, 0, 2)

### Reconstruct filling testset (fills missing values only, not valid for noisy data)

In [ ]:
recon_fill_test = copy.deepcopy(corr_test_x.cpu().numpy())
np.copyto(recon_fill_test, recon_test.cpu().numpy(), where = recon_fill_test==0)
recon_fill_test = torch.from_numpy(recon_fill_test)

### Interpolation

In [ ]:
mean_fill_test = copy.deepcopy(corr_test_x).detach().cpu().numpy()
for i in range(mean_fill_test.shape[0]):
    for j in range(mean_fill_test.shape[2]):
        if np.count_nonzero(mean_fill_test[i,:,j]) == 0: # when all data points in this channel are missing
            ch_mean = 0
        else:
            ch_mean = np.sum(mean_fill_test[i,:,j]) / np.count_nonzero(mean_fill_test[i,:,j])
        mean_fill_test[i, mean_fill_test[i,:,j] == 0, j] = ch_mean
mean_fill_test = torch.from_numpy(mean_fill_test)

### Linear Interpolation

In [ ]:
linear_interp_test = copy.deepcopy(corr_test_x).detach().cpu().numpy()
for i in range(linear_interp_test.shape[0]):
    for j in range(linear_interp_test.shape[2]):
        if np.count_nonzero(linear_interp_test[i,:,j]) == 0: # when all data points in this channel are missing
            linear_interp_test[i, :, j] = 0.0
        else:
            idxs = np.arange(linear_interp_test.shape[1]) # indexes of all the samples
            zero_filter = linear_interp_test[i,:,j] == 0 # index filter for zero values
            zero_idxs = idxs[zero_filter] # indexes for zero values
            non_zero_idxs = idxs[~zero_filter] # xp, indexes for non-zero values
            non_zero_vals = linear_interp_test[i, ~zero_filter, j] # fp, non-zero values
            interp_vals = np.interp(zero_idxs, non_zero_idxs, non_zero_vals) # interpolated values
            linear_interp_test[i,zero_idxs,j] = interp_vals # fill interpolated values to the corrupted signal
linear_interp_test = torch.from_numpy(linear_interp_test)

### Evaluate RMSE

In [ ]:
corr_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1).cpu().numpy(), corr_test_x.reshape(corr_test_x.shape[0],-1).cpu().numpy(), squared=False)
print('Corr RMSE:\n' + str(corr_rms))

In [ ]:
recon_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1).cpu().numpy(), recon_test.reshape(recon_test.shape[0],-1).cpu().numpy(), squared=False)
print('Recon RMSE:\n' + str(recon_rms))

In [ ]:
recon_fill_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1).cpu().numpy(), recon_fill_test.reshape(recon_fill_test.shape[0],-1).cpu().numpy(), squared=False)
print('Recon Fill RMSE:\n' + str(recon_fill_rms))

In [ ]:
mean_fill_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1), mean_fill_test.reshape(mean_fill_test.shape[0],-1).cpu().numpy(), squared=False)
print('Mean Fill RMSE:\n' + str(mean_fill_rms))

In [ ]:
linear_interp_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1), linear_interp_test.reshape(linear_interp_test.shape[0],-1).cpu().numpy(), squared=False)
print('Linear Interpolation RMSE:\n' + str(linear_interp_rms))

### Plot Data Sample

In [ ]:
data_id = 0

plt.imshow(test_x[data_id])
plt.title('Raw Data')
plt.show()

plt.imshow(corr_test_x[data_id])
plt.title('Corrupted')
plt.show()

plt.imshow(recon_test[data_id])
plt.title('Reconstructed')
plt.show()

plt.imshow(recon_fill_test[data_id])
plt.title('Recon Fill')
plt.show()

plt.imshow(mean_fill_test[data_id])
plt.title('Mean Fill')
plt.show()

plt.imshow(linear_interp_test[data_id])
plt.title('Linear Interpolation')
plt.show()

### Import HAR model

In [ ]:
from eval_har import *

In [ ]:
eval_har = get_eval_model(len(act_list), model_path='pamap2_cnn.pt').to(device)

### Evaluate HAR accuracy for raw data, corrupted data, and reconstructed data

In [ ]:
# Convert from N*171*27 into N*27*171
test_x = test_x.permute(0,2,1)
corr_test_x = corr_test_x.permute(0,2,1)
recon_test = recon_test.permute(0,2,1)
recon_fill_test = recon_fill_test.permute(0,2,1)
mean_fill_test = mean_fill_test.permute(0,2,1)
linear_interp_test = linear_interp_test.permute(0,2,1)

In [ ]:
# Extend one dummy dimension for CNN
raw_test_dataset = TensorDataset(test_x.reshape(test_x.shape[0], 1, test_x.shape[1], test_x.shape[2]), test_labels)
corr_test_dataset = TensorDataset(corr_test_x.reshape(corr_test_x.shape[0], 1, corr_test_x.shape[1], corr_test_x.shape[2]), test_labels)
recon_test_dataset = TensorDataset(recon_test.reshape(recon_test.shape[0], 1, recon_test.shape[1], recon_test.shape[2]), test_labels)
recon_fill_test_dataset = TensorDataset(recon_fill_test.reshape(recon_fill_test.shape[0], 1, recon_fill_test.shape[1], recon_fill_test.shape[2]), test_labels)
mean_fill_test_dataset = TensorDataset(mean_fill_test.reshape(recon_test.shape[0], 1, mean_fill_test.shape[1], mean_fill_test.shape[2]), test_labels)
linear_interp_test_dataset = TensorDataset(linear_interp_test.reshape(linear_interp_test.shape[0], 1, linear_interp_test.shape[1], linear_interp_test.shape[2]), test_labels)

In [ ]:
def evaluate_har(test_loader):
    correct = 0
    total = 0
    total_true = []
    total_pred = []
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)       
            # calculate outputs by running images through the network
            outputs = eval_har(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            # predicted = torch.argmax(outputs.data.cpu(), axis=1)
            total += labels.size(0)
            correct += (predicted == torch.argmax(labels, dim=1)).sum().item()
            
            total_pred = total_pred + predicted.cpu().numpy().tolist()
            total_true = total_true + (torch.argmax(labels, dim=1).cpu().numpy().tolist())
            
    print(f'Test Accuracy: {100.0 * correct / total} %')
    
    print(" | ".join(act_labels_txt))
    conf_mat = confusion_matrix(y_true = total_true, y_pred = total_pred)
    conf_mat = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
    print(np.array(conf_mat).round(3) * 100)
    
    return

In [ ]:
print("Raw testset:")
raw_test_loader = torch.utils.data.DataLoader(raw_test_dataset,
    batch_size=batch_size, shuffle=False)

evaluate_har(raw_test_loader)

In [ ]:
print("Corrupted testset:")
corr_test_loader = torch.utils.data.DataLoader(corr_test_dataset,
    batch_size=batch_size, shuffle=False)
evaluate_har(corr_test_loader)

In [ ]:
print("Reconstructed testset:")
recon_test_loader = torch.utils.data.DataLoader(recon_test_dataset,
    batch_size=batch_size, shuffle=False)
evaluate_har(recon_test_loader)

In [ ]:
print("Recon Fill testset:")
recon_fill_test_loader = torch.utils.data.DataLoader(recon_fill_test_dataset,
    batch_size=batch_size, shuffle=False)
evaluate_har(recon_fill_test_loader)

In [ ]:
print("Mean Fill testset:")
mean_fill_test_loader = torch.utils.data.DataLoader(mean_fill_test_dataset,
    batch_size=batch_size, shuffle=False)
evaluate_har(mean_fill_test_loader)

In [ ]:
print("Linear Interpolation testset:")
linear_interp_test_loader = torch.utils.data.DataLoader(linear_interp_test_dataset,
    batch_size=batch_size, shuffle=False)
evaluate_har(linear_interp_test_loader)

### Plot test data for each activity

In [ ]:
test_labels_np = torch.argmax(test_labels, dim=1).detach().cpu().numpy() # labels of testset, not one-hot
idx_list = np.arange(len(test_labels_np)) # list for indexes of all testset
rand_act_idxs = [] 
# randomly select a data sample for each activity
for act_id in range(len(act_list)):
    act_filter = test_labels_np[:] == act_id # f
    act_idx_list = idx_list[act_filter]
    rand_act_id = np.random.choice(act_list)
    rand_act_idxs.append(rand_act_id)

In [ ]:
fig = plt.figure(figsize=(50, 25))

for act_id in range(len(rand_act_idxs)):
    ax1 = fig.add_subplot(12,5,1+5*act_id)
    ax1.imshow(test_x[rand_act_idxs[act_id]])
    ax1.set_title('Raw Data - ' + act_labels_txt[act_id])
    
    ax2 = fig.add_subplot(12,5,2+5*act_id)
    ax2.imshow(corr_test_x[rand_act_idxs[act_id]])
    ax2.set_title('Corrupted - ' + act_labels_txt[act_id])
    
    ax3 = fig.add_subplot(12,5,3+5*act_id)
    ax3.imshow(recon_test[rand_act_idxs[act_id]])
    ax3.set_title('Reconstructed - ' + act_labels_txt[act_id])
    
    ax4 = fig.add_subplot(12,5,4+5*act_id)
    ax4.imshow(recon_fill_test[rand_act_idxs[act_id]] )
    ax4.set_title('Recon Fill - ' + act_labels_txt[act_id])

    ax5 = fig.add_subplot(12,5,5+5*act_id)
    ax5.imshow(mean_fill_test[rand_act_idxs[act_id]] )
    ax5.set_title('Mean Fill - ' + act_labels_txt[act_id])

### Transform the input timeseries to encoded latent vectors

In [ ]:
# z_run = vrae.transform(test_dataset)

# #If the latent vectors have to be saved, pass the parameter `save`
# # z_run = vrae.transform(dataset, save = True)

### Visualize using PCA and tSNE

In [ ]:
# plot_clustering(z_run, y_val, engine='matplotlib', download = False)

# # If plotly to be used as rendering engine, uncomment below line
# #plot_clustering(z_run, y_val, engine='plotly', download = False)